In [19]:
import json
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm_notebook
import random
import torch.optim as optim
from PIL import Image
from torchvision import models
import os, sys
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"
from tensorboardX import SummaryWriter


In [5]:
torch.__version__

'1.2.0'

In [6]:
class config():
    def __init__(self):
        self.data='../../dataset/train_images/VG_100K/'
        self.save='../checkpoints/'
        self.expname='vgg_8sa3_8t2i_ca1'
        self.expno=0
        self.pretrained_model='vgg'
        self.num_classes=None
        # training arguments
        self.epochs=100
        self.batchsize=64
        self.lr=0.0001
        self.wd=1e-4
        self.sparse=False
        self.optim='adam'
        # miscellaneous options
        self.seed=123
        self.cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.cuda else "cpu")
        self.pretrained_holistic=0

In [11]:
args = config()

In [14]:
args = config()
writer = SummaryWriter(comment = args.expname)
args.batchsize

64

In [17]:

torch.manual_seed(args.seed)
random.seed(args.seed)

if args.cuda:
    torch.cuda.manual_seed(args.seed)
    torch.backends.cudnn.benchmark = True

if not os.path.exists(args.save):
    os.makedirs(args.save)



In [18]:
def create_optimizer(args, parameters):
    if args.optim=='adam':
        optimizer   = optim.Adam(parameters, lr=args.lr, weight_decay=args.wd)
    elif args.optim=='adagrad':
        optimizer   = optim.Adagrad(parameters, lr=args.lr, weight_decay=args.wd)
    elif args.optim=='sgd':
        optimizer   = optim.SGD(parameters, lr=args.lr, weight_decay=args.wd)
    elif args.optim == 'adadelta':
        optimizer = optim.Adadelta(parameters, lr=args.lr, weight_decay=args.wd)
    return optimizer

In [21]:
pretrained_vgg16 = models.vgg(pretrained=True,)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /home/mshaikh2/.cache/torch/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:15<00:00, 36.3MB/s] 


In [25]:
vgg_features = pretrained_vgg16
vgg_features

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256